## DeepFM

In [2]:
import os
import numpy as np
import pandas as pd
from collections import namedtuple

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.preprocessing import  MinMaxScaler, LabelEncoder

from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report, roc_curve

##### 获取数据

In [3]:
def get_criteo_data():
    """ 读取criteo数据集 """
    train_path = '../data/criteo/train.txt'
    test_path = '../data/criteo/test.txt'

    dense_column_names = ['I' + str(i) for i in range(1, 14)]
    sparse_column_names = ['C' + str(i) for i in range(14, 40)]
    column_names = ['label'] + dense_column_names + sparse_column_names

    train_df = pd.read_csv(train_path, names=column_names, sep='\t')
    test_df = pd.read_csv(test_path, names=column_names, sep='\t')
    return train_df, test_df, dense_column_names, sparse_column_names

train_df, test_df, dense_column_names, sparse_column_names = get_criteo_data()
print('train.shape: {}, test.shape: {}'.format(train_df.shape, test_df.shape))

train.shape: (1000000, 40), test.shape: (1000000, 40)


##### 数据预处理

In [7]:
data = pd.concat([train_df, test_df], axis=0)

def data_processing(df, dense_column_names, sparse_column_names):
    """ 数据预处理 """
    df[dense_column_names] = df[dense_column_names].fillna(0.0)
    for f in dense_column_names:
        df[f] = df[f].apply(lambda x: np.log(x+1) if x > -1 else -1)
    
    df[sparse_column_names] = df[sparse_column_names].fillna("-1")
    for f in sparse_column_names:
        lbe = LabelEncoder()
        df[f] = lbe.fit_transform(df[f])
    return df[dense_column_names + sparse_column_names]

df = data_processing(data, dense_column_names, sparse_column_names)
df['label'] = data['label']

train_df, test_df = df.iloc[0:train_df.shape[0]], df.iloc[train_df.shape[0]:]
print('train.shape:{}, test.shape:{}'.format(train_df.shape, test_df.shape))

train.shape:(1000000, 40), test.shape:(1000000, 40)


##### 模型构建

In [8]:
SparseFeature = namedtuple('SparseFeature', ['name', 'vocabulary_size', 'embedding_size'])
DenseFeature = namedtuple('DenseFeature', ['name', 'dimension'])

feature_columns = [SparseFeature(f, vocabulary_size=df[f].nunique(), embedding_size=4) for f in sparse_column_names] + \
[DenseFeature(f, 1) for f in dense_column_names]

In [9]:
class FM_Layer(Layer):
    def __init__(self):
        super(FM_Layer, self).__init__()
        
    def call(self, inputs):
        concat_embed_values = inputs
        print('concat_embed_values.shape: ', concat_embed_values.shape) # (None, 26, 4)
        sum_square = tf.square(tf.reduce_sum(concat_embed_values, axis=1, keepdims=True)) # (None, 1, 4)
        print('sum_square.shape: ', sum_square.shape)
        square_sum = tf.reduce_sum(concat_embed_values * concat_embed_values, axis=1, keepdims=True) # (None, 1, 4)
        print('square_sum.shape: ', square_sum.shape)
        output = sum_square - square_sum # 和的平方-平方的和
        output = 0.5 * tf.reduce_sum(output, axis=2, keepdims=False) # (None 1)
        print('output.shape: ', output.shape)
        return output
    
    def compute_output_shape(self, input_shape):
        return (None, 1)
    
def model_metric(prob, label, thr=0.5):
    """ 模型评估 """
    # AUC
    fpr, tpr, threshold = metrics.roc_curve(label, prob)
    auc = metrics.auc(fpr, tpr)
    score = metrics.accuracy_score(label, prob > thr)
    # LogLoss
    logloss = log_loss(label, prob)
    print('模型准确率:{}, AUC得分:{}, LogLoss:{}'.format(score, auc, logloss))
    print(classification_report(label, prob > thr, digits=2))
    print('==========================================================')

def build_input_layers(feature_columns):
    """ 构建输入层 """
    dense_input_dict, sparse_input_dict = {}, {}
    for f in feature_columns:
        if isinstance(f, DenseFeature):
            dense_input_dict[f.name] = Input(shape=(f.dimension, ), name=f.name)
        elif isinstance(f, SparseFeature):
            sparse_input_dict[f.name] = Input(shape=(1, ), name=f.name)
    return dense_input_dict, sparse_input_dict
    
def build_embedding_layers(feature_columns, is_linear):
    """ 构建embedding层 """
    embedding_layer_dict = {}
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    if is_linear:
        for f in sparse_feature_columns:
            embedding_layer_dict[f.name] = Embedding(f.vocabulary_size+1, 1, name='1d_emb_' + f.name)
    else:
        for f in sparse_feature_columns:
            embedding_layer_dict[f.name] = Embedding(f.vocabulary_size+1, f.embedding_size, name='kd_emb_' + f.name)
    return embedding_layer_dict
    
def get_linear_logits(dense_input_dict, sparse_input_dict, feature_columns):
    """ 数值特征拼接一起传入全连接层 + 类别特征onehot，flatten，add """
    concat_dense_inputs = Concatenate(axis=1)(list(dense_input_dict.values()))
    dense_logits_output = Dense(1)(concat_dense_inputs)
    
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    embedding_layer_dict = build_embedding_layers(sparse_feature_columns, is_linear=True)
    
    # embedding(input)查表操作，返回对应input的嵌入向量
    sparse_1d_embed_list = []
    for f in sparse_feature_columns:
        _input = sparse_input_dict[f.name]
        _embed = Flatten()(embedding_layer_dict[f.name](_input))
        sparse_1d_embed_list.append(_embed)
    
    sparse_logits_output = Add()(sparse_1d_embed_list)
    linear_logits = Add()([dense_logits_output, sparse_logits_output])
    return linear_logits
    
def get_fm_logits(sparse_input_dict, kd_embedding_layer_dict, feature_columns):
    """ 取出input所对应的嵌入向量拼接在一起，计算和的平和-平方的和 """
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    sparse_kd_embed_list = []
    for f in sparse_feature_columns:
        _input = sparse_input_dict[f.name]
        _embed = kd_embedding_layer_dict[f.name](_input)
        sparse_kd_embed_list.append(_embed)
    
    concat_sparse_kd_embed_list = Concatenate(axis=1)(sparse_kd_embed_list)
    fm_logits = FM_Layer()(concat_sparse_kd_embed_list)
    return fm_logits
    
def get_dnn_logits(sparse_input_dict, kd_embedding_layer_dict, feature_columns):
    sparse_feature_columns = list(filter(lambda x: isinstance(x, SparseFeature), feature_columns))
    sparse_kd_embed_list = []
    for f in sparse_feature_columns:
        _input = sparse_input_dict[f.name]
        _embed = kd_embedding_layer_dict[f.name](_input)
        flatten_embed = Flatten()(_embed)
        sparse_kd_embed_list.append(flatten_embed)
    concat_sparse_kd_embed = Concatenate(axis=1)(sparse_kd_embed_list)
    print('concat_sparse_kd_embed.shape: ', concat_sparse_kd_embed.shape)
    
    # DNN
    dnn_out = Dropout(0.5)(Activation(activation='relu')(BatchNormalization()(Dense(4)(concat_sparse_kd_embed))))
    #dnn_out = Dropout(0.2)(Activation(activation='relu')(BatchNormalization()(Dense(4)(dnn_out))))
    #dnn_out = Dropout(0.5)(Activation(activation='relu')(Dense(4, activation='relu')(concat_sparse_kd_embed))
    #dnn_out = Dropout(0.5)(Dense(32, activation='relu')(dnn_out))
    dnn_out = Dropout(0.5)(Dense(8, activation='relu')(dnn_out))
    dnn_logits = Dense(1)(dnn_out)
    return dnn_logits
    
def DeepFM(feature_columns, seed=1024, l2_reg=1e-5, task='binary'):
    """ Instantiates FM architecture
    :param feature_columns 
    :param seed
    :param l2_reg L2regularization
    :return: A kears model instance
    """
    dense_input_dict, sparse_input_dict = build_input_layers(feature_columns)
    input_list = list(dense_input_dict.values()) + list(sparse_input_dict.values())
    
    # linear w * x + b
    linear_logits = get_linear_logits(dense_input_dict, sparse_input_dict, feature_columns)
    
    # fm 0.5 * [sum(vixi)**2 - sum(vixi*vixi)
    kd_embedding_layer_dict = build_embedding_layers(feature_columns, is_linear=False)
    fm_logits = get_fm_logits(sparse_input_dict, kd_embedding_layer_dict, feature_columns)
    
    # dnn next_a = σ(w * a + b)
    dnn_logits = get_dnn_logits(sparse_input_dict, kd_embedding_layer_dict, feature_columns)
    
    #output_logits = Add()([linear_logits])
    #output_logits = Add()([linear_logits, fm_logits])
    output_logits = Add()([linear_logits, fm_logits, dnn_logits])
    output_layer = Activation("sigmoid")(output_logits)
    model = Model(input_list, output_layer)
    return model

model = DeepFM(feature_columns)
model.summary()

concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
concat_sparse_kd_embed.shape:  (None, 104)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
C14 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C15 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C16 (InputLayer)                [(None, 1)]          0                                            
__________________________________________________________________________________________________
C17 (InputLayer)   

                                                                 C28[0][0]                        
__________________________________________________________________________________________________
kd_emb_C29 (Embedding)          (None, 1, 4)         1487736     C29[0][0]                        
                                                                 C29[0][0]                        
__________________________________________________________________________________________________
kd_emb_C30 (Embedding)          (None, 1, 4)         44          C30[0][0]                        
                                                                 C30[0][0]                        
__________________________________________________________________________________________________
kd_emb_C31 (Embedding)          (None, 1, 4)         17028       C31[0][0]                        
                                                                 C31[0][0]                        
__________

In [10]:
# 模型训练
train_input = {f: train_df[f] for f in dense_column_names + sparse_column_names}
test_input = {f: test_df[f] for f in dense_column_names + sparse_column_names}

my_callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=2, mode='auto')
]
model.compile('adam',
              loss='binary_crossentropy',
              metrics=["binary_crossentropy", tf.keras.metrics.AUC(name='auc')])
model.fit(train_input,
          train_df['label'].values,
          batch_size=1024,
          epochs=100,
          validation_split=0.2,
          callbacks=my_callbacks)

Epoch 1/100
concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
782/782 [==============================] - ETA: 0s - loss: 0.5987 - binary_crossentropy: 0.5987 - auc: 0.6940concat_embed_values.shape:  (None, 26, 4)
sum_square.shape:  (None, 1, 4)
square_sum.shape:  (None, 1, 4)
output.shape:  (None, 1)
782/782 [==============================] - 57s 66ms/step - loss: 0.5987 - binary_crossentropy: 0.5987 - auc: 0.6940 - val_loss: 0.4912 - val_binary_crossentropy: 0.4912 - val_auc: 0.7524
Epoch 2/100
782/782 [==============================] - 51s 65ms/step - loss: 0.4394 - binary_crossentropy: 0.4394 - auc: 0.8134 - val_loss: 0.4807 - val_binary_crossentropy: 0.4807 - val_auc: 0.7638
Epoch 3/100
782/782 [==============================] - 51s 66ms/step - loss: 0.3702 - binary_crossen

In [11]:
# 模型预测与评估
result = model.predict(test_input)
model_metric(np.array([i[0] for i in result]), test_df['label'].values)

concat_embed_values.shape:  (32, 26, 4)
sum_square.shape:  (32, 1, 4)
square_sum.shape:  (32, 1, 4)
output.shape:  (32, 1)
模型准确率:0.746038, AUC得分:0.7074822674693234, LogLoss:0.7771902693858965
              precision    recall  f1-score   support

           0       0.82      0.85      0.83    751819
           1       0.49      0.43      0.45    248181

    accuracy                           0.75   1000000
   macro avg       0.65      0.64      0.64   1000000
weighted avg       0.74      0.75      0.74   1000000

